In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.datasets import mnist

class NeuralNet:
    def __init__(self):
        # Load MNIST dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        self.X_train = X_train.reshape(-1, 28 * 28).astype('float32') / 255.0
        self.X_test = X_test.reshape(-1, 28 * 28).astype('float32') / 255.0
        self.y_train = y_train
        self.y_test = y_test

    def preprocess(self):
        #Preprocess the data: standardization, normalization, etc.
        # StandardScaler not needed as data is already in [0, 1] range.
        self.y_train = tf.keras.utils.to_categorical(self.y_train, num_classes=10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, num_classes=10)

    def build_model(self, input_dim, num_hidden_layers, activation, learning_rate, dropout_rate=0.3):
        #Build and compile a Keras sequential model.
        model = Sequential()
        model.add(Input(shape=(input_dim,)))
        for _ in range(num_hidden_layers):
            model.add(Dense(input_dim, activation=activation))
            model.add(Dropout(dropout_rate))
            model.add(BatchNormalization())
        model.add(Dense(10, activation='softmax'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

    def train_evaluate(self):
        # Train and evaluate the model with different configurations.
        X_train, X_test = self.X_train, self.X_test
        y_train, y_test = self.y_train, self.y_test

        # Hyperparameters
        activations = ['sigmoid', 'relu', 'tanh']
        epochs = [10, 50]
        num_hidden_layers_options = [0, 1, 2]  # Number of hidden layers
        learning_rates = [0.01, 0.001]

        results = []

        # Create "Graphs" directory if it doesn't exist
        if not os.path.exists("Graphs"):
            os.makedirs("Graphs")

        for activation in activations:
            for epoch in epochs:
                for num_hidden_layers in num_hidden_layers_options:
                    fig, ax = plt.subplots(figsize=(10, 5))
                    for learning_rate in learning_rates:
                        model = self.build_model(input_dim=X_train.shape[1], num_hidden_layers=num_hidden_layers, activation=activation, learning_rate=learning_rate)

                        history = model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_test, y_test), verbose=0)

                        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
                        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

                        results.append({
                            'Activation': activation,
                            'Learning Rate': learning_rate,
                            'Epochs': epoch,
                            'Number of Hidden Layers': num_hidden_layers,
                            'Training Accuracy (%)': round(train_acc * 100, 2),
                            'Testing Accuracy (%)': round(test_acc * 100, 2),
                            'Training Loss (%)': round(train_loss * 100, 2),
                            'Testing Loss (%)': round(test_loss * 100, 2)
                        })

                        label_suffix = f'LR={learning_rate}'
                        ax.plot(history.history['accuracy'], label=f'Train Acc ({label_suffix})')
                        ax.plot(history.history['val_accuracy'], label=f'Val Acc ({label_suffix})')

                    ax.set_title(f'{activation.capitalize()}, {epoch} epochs, {num_hidden_layers} hidden layers')
                    ax.set_ylabel('Accuracy (%)')
                    ax.set_xlabel('Epoch')
                    ax.legend(loc='lower right')
                    ax.set_ylim(0, 1)

                    # Save the figure
                    plt.tight_layout()
                    fig.savefig(f'Graphs/{activation}_{epoch}epochs_{num_hidden_layers}hiddenlayers.png')
                    plt.close(fig)

        results_df = pd.DataFrame(results)
        display(results_df)  # Use display() to show DataFrame in a Jupyter Notebook

# Execution in a Jupyter Notebook cell
neural_net = NeuralNet()
neural_net.preprocess()
neural_net.train_evaluate()
